# Text to image via bitmap tests

## This here doesn't work

In [ ]:
from __future__ import print_function
import string
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import numpy as np

def char_to_pixels(text, path='arialbd.ttf', fontsize=14):
    """
    Based on https://stackoverflow.com/a/27753869/190597 (jsheperd)
    """
    font = ImageFont.truetype(path, fontsize) 
    w, h = font.getsize(text)  
    h *= 2
    image = Image.new('L', (w, h), 1)  
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), text, font=font) 
    arr = np.asarray(image)
    arr = np.where(arr, 0, 1)
    arr = arr[(arr != 0).any(axis=1)]
    return arr

def display(arr):
    result = np.where(arr, '#', ' ')
    print('\n'.join([''.join(row) for row in result]))

for c in string.ascii_uppercase:
    arr = char_to_pixels(
        c, 
        path='/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf', 
        fontsize=9)
    print(arr.shape)
    display(arr)
    print()

AttributeError: 'FreeTypeFont' object has no attribute 'getsize'

## Hunting for info

In [22]:
font = ImageFont.truetype("arial", 9) 
type(font)
help(font)

Help on FreeTypeFont in module PIL.ImageFont object:

class FreeTypeFont(builtins.object)
 |  FreeTypeFont(font: 'StrOrBytesPath | BinaryIO', size: 'float' = 10, index: 'int' = 0, encoding: 'str' = '', layout_engine: 'Layout | None' = None) -> 'None'
 |
 |  FreeType font wrapper (requires _imagingft service)
 |
 |  Methods defined here:
 |
 |  __getstate__(self) -> 'list[Any]'
 |      Helper for pickle.
 |
 |  __init__(self, font: 'StrOrBytesPath | BinaryIO', size: 'float' = 10, index: 'int' = 0, encoding: 'str' = '', layout_engine: 'Layout | None' = None) -> 'None'
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __setstate__(self, state: 'list[Any]') -> 'None'
 |
 |  font_variant(self, font: 'StrOrBytesPath | BinaryIO | None' = None, size: 'float | None' = None, index: 'int | None' = None, encoding: 'str | None' = None, layout_engine: 'Layout | None' = None) -> 'FreeTypeFont'
 |      Create a copy of this FreeTypeFont object,
 |      using any specified a

Looks like getsize is not a thing but this looks good:

```
 |  getbbox(self, text: 'str | bytes', mode: 'str' = '', direction: 'str | None' = None, features: 'list[str] | None' = None, language: 'str | None' = None, stroke_width: 'float' = 0, anchor: 'str | None' = None) -> 'tuple[float, float, float, float]'
 |      Returns bounding box (in pixels) of given text relative to given anchor
 |      when rendered in font with provided direction, features, and language.
 [...]
 |      :return: ``(left, top, right, bottom)`` bounding box
```

Also check out the `getmask` functions later because they just might be exactly what we're looking for here

In [25]:
from pprint import pprint as pp
pp(font.getbbox("plop"))
pp(font.getbbox("1"))
pp(font.getbbox("anticonstitutionnellement"))

(0, 2, 17, 11)
(0, 2, 5, 9)
(0, 2, 103, 9)


Hold tf up how do we get the list of available fonts, custom fonts (and even common ones like consolas) don't work

In [26]:
import os
os.listdir('C:\\Windows\\Fonts')

['8514fix.fon',
 '8514fixe.fon',
 '8514fixg.fon',
 '8514fixr.fon',
 '8514fixt.fon',
 '8514oem.fon',
 '8514oeme.fon',
 '8514oemg.fon',
 '8514oemr.fon',
 '8514oemt.fon',
 '8514sys.fon',
 '8514syse.fon',
 '8514sysg.fon',
 '8514sysr.fon',
 '8514syst.fon',
 '85775.fon',
 '85855.fon',
 '85f1255.fon',
 '85f1256.fon',
 '85f1257.fon',
 '85f874.fon',
 '85s1255.fon',
 '85s1256.fon',
 '85s1257.fon',
 '85s874.fon',
 'Alef-Bold.ttf',
 'Alef-Regular.ttf',
 'Amiri-Bold.ttf',
 'Amiri-BoldItalic.ttf',
 'Amiri-Italic.ttf',
 'Amiri-Regular.ttf',
 'AmiriQuran.ttf',
 'app775.fon',
 'app850.fon',
 'app852.fon',
 'app855.fon',
 'app857.fon',
 'app866.fon',
 'app932.fon',
 'app936.fon',
 'app949.fon',
 'app950.fon',
 'arial.ttf',
 'arialbd.ttf',
 'arialbi.ttf',
 'ariali.ttf',
 'ariblk.ttf',
 'bahnschrift.ttf',
 'c8514fix.fon',
 'c8514oem.fon',
 'c8514sys.fon',
 'Caladea-Bold.ttf',
 'Caladea-BoldItalic.ttf',
 'Caladea-Italic.ttf',
 'Caladea-Regular.ttf',
 'calibri.ttf',
 'calibrib.ttf',
 'calibrii.ttf',
 'calib

## So let's redo the code that doesn't work

In [45]:
input_string: str = "test"

def getTextWidthAndHeight(font: ImageFont.FreeTypeFont, text: str) -> tuple:
    bb = font.getbbox(text)
    return (bb[2]-bb[0]), (bb[3]-bb[1])

font = ImageFont.truetype('arial', 12) 

w, h = getTextWidthAndHeight(font,input_string)
h *= 2
image = Image.new('L', (w, h), 1)  
draw = ImageDraw.Draw(image)
draw.text((0, 0), input_string, font=font) 
arr = np.asarray(image)
pp(arr)


array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1,

In [46]:
arr = np.where(arr, 0, 1)
arr = arr[(arr != 0).any(axis=1)]
pp(arr)

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0]])


**It works**

Now to make it usable for us we need to have it as white on black: turn all zeroes to an arbitrary "white" value and turn the ones into zeroes (black)

In [47]:
inverted_arr = arr ^ 1
pp(inverted_arr)

array([[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1]])


In [50]:
whitevalue: int = 255
white_arr = inverted_arr * whitevalue
pp(white_arr, width=200)

array([[255,   0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255,   0, 255, 255],
       [255,   0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255,   0, 255, 255],
       [  0,   0,   0, 255, 255,   0,   0,   0, 255, 255, 255,   0,   0,
          0, 255, 255,   0,   0,   0, 255],
       [255,   0, 255, 255,   0, 255, 255,   0,   0, 255, 255,   0, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255,   0, 255, 255,   0, 255,
        255, 255, 255, 255,   0, 255, 255],
       [255,   0, 255,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,
          0,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255, 255, 255, 255, 255, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255,   0, 255, 255,   0, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0,   0, 255, 255,   0,   0,   0, 255, 255, 255,  

## Put it all together

In [ ]:
import numpy as np
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

def _getTextWidthAndHeight(font: ImageFont.FreeTypeFont, text: str) -> tuple:
    bb = font.getbbox(text)
    return (bb[2]-bb[0]), (bb[3]-bb[1])

def get_drawn_text_bitmap(text,fontfam='arial',fontsize=12) -> np.ndarray:
    font = ImageFont.truetype(fontfam, fontsize) 
    w, h = getTextWidthAndHeight(font,input_string)
    h *= 2
    image = Image.new('L', (w, h), 1)  
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), input_string, font=font) 
    arr = np.asarray(image)
    arr = np.where(arr, 0, 1)
    arr = arr[(arr != 0).any(axis=1)]
    white_on_black_arr = (arr ^ 1) * whitevalue
    return white_on_black_arr
    

get_drawn_text_bitmap("This is a text bitmap")



array([[255,   0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255,   0, 255, 255],
       [255,   0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255,   0, 255, 255],
       [  0,   0,   0, 255, 255,   0,   0,   0, 255, 255, 255,   0,   0,
          0, 255, 255,   0,   0,   0, 255],
       [255,   0, 255, 255,   0, 255, 255,   0,   0, 255, 255,   0, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255,   0, 255, 255,   0, 255,
        255, 255, 255, 255,   0, 255, 255],
       [255,   0, 255,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,
          0,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255, 255, 255, 255, 255, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255,   0, 255, 255,   0, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0,   0, 255, 255,   0,   0,   0, 255, 255, 255,  

Also we're going to need the size of the array in X and Y and I have a brainfart on how to do that

In [63]:
bm = get_drawn_text_bitmap("This is a loooooooooooooooooooooooooooooooooooong but short text bitmap")
pp(bm)
pp(len(bm))
pp(len(bm[0]))
pp(np.shape(bm))

array([[255,   0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255,   0, 255, 255],
       [255,   0, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255,   0, 255, 255],
       [  0,   0,   0, 255, 255,   0,   0,   0, 255, 255, 255,   0,   0,
          0, 255, 255,   0,   0,   0, 255],
       [255,   0, 255, 255,   0, 255, 255,   0,   0, 255, 255,   0, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255,   0, 255, 255,   0, 255,
        255, 255, 255, 255,   0, 255, 255],
       [255,   0, 255,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,
          0,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255, 255, 255, 255, 255, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0, 255, 255,   0, 255, 255, 255,   0, 255, 255,   0, 255,
        255,   0, 255, 255,   0, 255, 255],
       [255,   0,   0, 255, 255,   0,   0,   0, 255, 255, 255,  

# So uh, about that getmask thing ?

In [64]:
font = ImageFont.truetype("arial", 9) 
help(font.getmask2)

Help on method getmask2 in module PIL.ImageFont:

getmask2(text: 'str | bytes', mode: 'str' = '', direction: 'str | None' = None, features: 'list[str] | None' = None, language: 'str | None' = None, stroke_width: 'float' = 0, anchor: 'str | None' = None, ink: 'int' = 0, start: 'tuple[float, float] | None' = None, *args: 'Any', **kwargs: 'Any') -> 'tuple[Image.core.ImagingCore, tuple[int, int]]' method of PIL.ImageFont.FreeTypeFont instance
    Create a bitmap for the text.

    If the font uses antialiasing, the bitmap should have mode ``L`` and use a
    maximum value of 255. If the font has embedded color data, the bitmap
    should have mode ``RGBA``. Otherwise, it should have mode ``1``.

    :param text: Text to render.
    :param mode: Used by some graphics drivers to indicate what mode the
                 driver prefers; if empty, the renderer may return either
                 mode. Note that the mode is always a string, to simplify
                 C-level implementations.

  

Lol nope, not what we're looking for